#### Run this cell first to import libraries and set variables.
Some code cells may depend on the previous code cells. It is recommanded to run the cells one by one downwards.

In [ ]:
%matplotlib inline  
from GEOS2115_utils import *

#set start time, end time and time step 
start_time = 0
end_time = 40
time_step = 10

#### Topography Maps

In [ ]:
for time in range(start_time, end_time+1, time_step):
    draw_topography_map(time)

#### Run the model. It will take a while to finish, be patient ...

In [ ]:
run_model(start_time, end_time, time_step)

#### Temperature Map

In [ ]:
air_or_ocean = 'air' 
#air_or_ocean = 'ocean'
time = 0

draw_NASA_present_day_temperature_map()

draw_temperature_map(time, air_or_ocean)


#### Precipitation Map

In [ ]:
time = 0 

draw_NASA_present_day_precipitation_map()

draw_precipitation_map(time)

#### Surface Wind Velocity Map

In [ ]:
time = 0

draw_surfacre_wind_velocity_map(time)

#### Ocean Current Velocity Map

In [ ]:
time = 0

draw_ocean_current_velocity_map(time)

#### Salinity Map

In [ ]:
time = 0

draw_salinity_map(time)

#### Upwelling Map

In [ ]:
time = 0 

draw_upwelling_map(time)

#### this cell will draw a line plot to show the temperature change over time at a given location 

In [ ]:
lon = 20
lat = 35
property_name = 'temperature'
#property_name = 'precipitation'
#property_name = 'air_velocity'
#property_name = 'ocean_velocity'

draw_plot_through_time(property_name, lon, lat, start_time, end_time, time_step)

#### xy plots of transects

In [ ]:
time = 0
lon = 45
#property_name = 'temperature'
#property_name = 'ocean_temperature'
#property_name = 'salinity'
property_name = 'ocean_velocity'

draw_transects(property_name, time, lon)